# SQL Data Query Language Queries - One Page

- A one page reference for common Data Query Language Commands, all in one place.  
- My own query aims and notes are included. 
- The database tables uses, unless specified, are from the BikeStores sample database - see README.

**CONTENTS**

**1. SQL Query Order**

**2. Sorting**

**3. Limiting**

**4. Filtering**

**5. Joining Tables**

**6. Grouping**

**7. SubQuery**

**8. Set Operators**

**9. Common Table Expressions**

**10. Pivot**

**11. Expressions - Coalesce, Case, NullIf**

# 1. SQL Query Order

**SQL queries are evaluated in the following order:**

**FROM** - choose and join tables

**WHERE** - filters the data

**GROUP BY** - aggregates the data

**HAVING** - filters the aggregated data

**SELECT** - returns the final data

**ORDER BY** - sorts the final data

**LIMIT** - limits sorted data to a row count

-----

# 2. SORTING

## 2a. Order By

**QUERY AIM:**
- This query retrieves all first names and orders them in descending order.

```SQL
SELECT 
    first_name
FROM
    sales.customers
ORDER BY 
    first_name DESC
```

--

**QUERY AIM:**
- This query retrieves first names and then orders them first by city and then by first name.

```SQL
SELECT 
    first_name
FROM
    sales.customers
ORDER BY 
    city,
    first_name
```

--

**QUERY AIM:**
- This query returns all first names, ordered by the length of the name in characters.
- We can order results by expressions as well as columns.

```SQL
SELECT
    first_name
FROM
    sales.customers
ORDER BY 
    LEN(first_name) DESC
```

-----

# 3. LIMITING

## 3a. Offset, Fetch

**QUERY AIM:**
- In this query, from a 20 team league, we want to skip the top 4 teams (OFFSET) and then retrieve the next 13 rows.

**NOTES:**
- Query order is very important here. SELECT is evaluated AFTER ORDER BY, so the teams are ordered by points first before we use OFFSET and FETCH.
- The table used in this query is an invented sports league table with team_name and points column.

```SQL
SELECT 
    team_name,
    points,
FROM 
    prem_table
ORDER BY 
    points DESC
OFFSET 4 ROWS
FETCH NEXT 13 ROWS ONLY
```

--

## 3b. Select Top

**QUERY AIMS:**
- The queries below would retrieve the TOP 10, the TOP 1% and the TOP 3 WITH TIES.

**NOTES:**
- TOP 3 WITH TIES means that all joint equal 3rd highest (or lowest) of a value are included in the result.

```SQL
SELECT TOP 10
SELECT TOP 1 PERCENT
SELECT TOP 3 WITH TIES
```

----

# 4. FILTERING

## 4a. Distinct

**QUERY AIM:**
- This query retuns all unique countries in the customers table.

```SQL
SELECT DISTINCT 
    country
FROM 
    sales.customers
```

--

## 4b. And, Or, In

**QUERY AIM:**
- This query uses OR and AND keywords to retrieve product details where the brand_id is 1 or 2, AND the list_price is higher than 40.

**NOTES:**
- AND is evaluated before OR
- Brackets are important!
- Without brackets, the query would return (brand_id = 2 AND list_price > 40)

```SQL
SELECT 
    product_name,
    brand_id,
    list_price
FROM 
    production.products
WHERE
    (brand_id = 1 OR brand_id = 2)
AND
    list_price > 40
```

--

**QUERY AIM:**
- This query uses the IN operator to find products with a brand_id that is either 1,2,3 or 4.

```SQL
SELECT 
    product_id,
    brand_id
FROM
    products
WHERE
    brand_id IN(1,2,3,4)
```

--

## 4c. Between, Not Between

**QUERY AIM:**
- This query uses NOT BETWEEN to find products whose list_price doesn't fall between 149.99 and 199.99 (inclusive).

```SQL
SELECT
    product_id,
    product_name,
    list_price
FROM
    production.products
WHERE
    list_price NOT BETWEEN 149.99 AND 199.99
```

--

**QUERY AIM:**
- This query finds all orders where the date of the order matches a specific date range.

**NOTES:**
- Note how the date is input as a string with 'YYYMMDD' format.

```SQL
SELECT 
    order_id,
    order_date
FROM
    orders
WHERE
    order_date BETWEEN '20170115' AND '20170117
```

--

## 4d. Like

**QUERY AIMS:**
- These queries use types of regular expressions to return filtered string results.

**NOTES:**
- 'z%' means starts with z and followed by ANY characters.
- '%er' means ANY characters followed by 'er' at the end.
- 't%s' means a string starting with 't', ending with 's' and ANY characters in between.
- '_u%' -> the underscore means 1 SINGLE character (ANY) followed here by a 'u' and then any characters, eg. Duncan, super, Humphrey
- '[ZY]%' -> Square brackets means either of these here (Z or Y) followed by ANY characters (%).
- '[^A-Z]%' -> The ^ sign means not any character in the range (A-Z) followed by ANY characters (%).
- If we want to include regex symbols such as '%^' in our search, then we can escape them by choosing any key we want to use before a regex character, then declaring that character with the ESCAPE keyword.

```SQL
WHERE
    last_name LIKE 'z%'
```

```SQL
WHERE
    last_name LIKE '%er'
```

```SQL
WHERE
    last_name LIKE 't%s'
```

```SQL
WHERE
    last_name LIKE '_u%'
```

```SQL
WHERE
    last_name LIKE '[ZY]%'
```

```SQL
WHERE
    last_name LIKE '[^A-X]%'
```

```SQL
WHERE 
    comment LIKE '%30!%%' ESCAPE '!'
```

--

## 4e. Column Aliases

**QUERY AIMS:**
- The queries below all use table aliases.
- The second query uses ' ' as we want to include a column alias name which has spaces.
- The third query is an example of using aliases within joins to make join queries easier to read.

```SQL
SELECT
    first_name + ' ' + last_name AS full_name
```

--

```SQL
SELECT
    first_name + ' ' + last_name AS 'Full Name'
```


--

**NOTES ON SPECIFYING COLUMNS:**
- In SQL, when performing a join, we don't need to specify which table a column belongs to, if the column is unique.
- In this example, we need to specify that we want the customer_id from table 'c' because their is also a customer_id column in 'o'.
- first_name is only unique to customers, so we don't need to specify 'c.first_name'.

```SQL
SELECT
    c.customer_id,
    first_name,
FROM
    sales.customers c
INNER JOIN sales.orders o ON o.customer_id = c.customer_id
```

------

# 5. Joining Tables

## 5a. Inner Join

**QUERY AIM:**
- This query retrieves all candidates who are also employees.

```SQL
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employeed_id,
    e.full_name employee_name
FROM 
    candidates c
    INNER JOIN 
        employees e
        ON c.fullname = e.fullname
```

--

**QUERY AIM:**
- This query retrieves product information and the category name for each product.

**NOTES:**
- An INNER JOIN finds matches between the product and category tables.
- Here, the category_id from the products table is used to get extra information about the category from the category table.

```SQL
SELECT 
    product_name, 
    category_name,
    list_price
FROM 
    production.products p
    INNER JOIN production.categories c
        ON p.category_id = c.category_id
```

--

## 5b. Left Join

**QUERY AIM:**
- This query returns ALL candidates (left table) and their name if they also appear in the employees table.
- NULL is returned in the employeed_id and employee_name columns if they don't appear in the employees table.

```SQL
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    candidates c
    LEFT JOIN employees e 
        ON e.fullname = c.fullname
```

--

**QUERY AIM:**
- This query returns ALL product names (left table) and their order_id.
- NULL will be returned if there is no order_id for that product.

**NOTES:**
- We can find out which products have NOT been ordered so far by viewing or filtering the NULL values.

```SQL
SELECT
    product_name,
    order_id
FROM
    production.products p
    LEFT JOIN sales.order_items o 
        ON o.product_id = p.product_id
    ORDER BY
        order_id;
```

--

## 5c. Right Join

**QUERY AIM:**
- This query retrieves all employees (right table) and any matches that appear in the candidates table.
- The query will return NULL in the candidates columns (left table) if there is no match.

**NOTES:** 
- This is the reverse of the LEFT JOIN.

```SQL
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employee_id,
    e.full_name employee_name
FROM 
    candidates c
    RIGHT JOIN employees e
        ON c.full_name = e.full_name
```

--

## 5d. Full Outer Join

**QUERY AIM:**
- This query retrieves all candidates and employees from both tables and shows us any matches where people appear in both tables.
- The query will return NULL in the employees 'side' or NULL in the candidates 'side' if there is no match.

**NOTES:**
- A FULL OUTER JOIN combines both a LEFT JOIN and RIGHT JOIN and gives us all matches and all NON matches from the two tables. 
- This type of join will always show all results from both tables.

```SQL
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    hr.candidates c
    FULL JOIN hr.employees e 
        ON e.fullname = c.fullname;
```

--

## 5e. Cross Join

**QUERY AIM:**
- This query aims to combine all possible combinations of store_id and product_id.
- This query can then be used as a left table in a left join where the store_id and product_id can be used to find matches in a sales table containing store_id and product_id information.

**NOTES:**
- CROSS JOIN isn't often used on its own but can be part of a query.

```SQL
SELECT
    s.store_id,
    p.product_id,
    ISNULL(sales, 0) sales
FROM
    sales.stores s
CROSS JOIN production.products p
    LEFT JOIN (....SUB QUERY...)

```

--

## 5f. Self Join

**QUERY AIM:**
- This query aims to get all staff emails alongside the email of the manager for each staff member. 
- This query sees the staffs table queried against itself.

**NOTES:**
- SELF JOIN is often used to query hierarchical data stored by ids in the same table.
- We can also include NULL in the results to find the 'top boss' who has NULL for manager_id.
- This is acheived by using a LEFT JOIN which will give us the email for EVERY staff member, regardless if they have a manager eg. the top boss.

```SQL
SELECT
    e.email employee_email,
    m.email manager_email
FROM 
    sales.staffs e
    LEFT JOIN sales.staffs m
    ON e.manager_id = m.staff_id;
```

-----

# 6. Grouping

## 6a. Group By

**QUERY AIM:**
- This query aims to return the total number of orders placed, grouped by customer_id and year, but only where the customer_id is either 1 or 2.

```SQL
SELECT
    customer_id,
    YEAR (order_date) order_year,
    COUNT (order_id) orders_placed
FROM
    sales.orders
WHERE
    customer_id IN (1, 2)
GROUP BY
    customer_id,
    YEAR (order_date)
ORDER BY
    customer_id

```

--

**QUERY AIM:**
- This query retrieves the total number of customers in each city.

```SQL
SELECT
    city,
    COUNT (customer_id) customer_count
FROM
    sales.customers
GROUP BY
    city
ORDER BY
    city
```

--

**QUERY AIM:**
- This query retrieves the average list price of products for each brand, where the model year is 2018.

**NOTES:**
- We use a join here because we only have brand_id in the products table and we retrieve the brand name from the brands table.

```SQL
SELECT
    brand_name,
    AVG (list_price) avg_price
FROM
    production.products p
INNER JOIN production.brands b ON b.brand_id = p.brand_id
WHERE
    model_year = 2018
GROUP BY
    brand_name
ORDER BY
    brand_name
```

--

**QUERY AIM:**
- This query retrieves the net value of all sales (after discount) for each order_id.

```SQL
SELECT
    order_id,
    SUM (
        quantity * list_price * (1 - discount)
    ) net_value
FROM
    sales.order_items
GROUP BY
    order_id;

```

--

## 6b. Having

**QUERY AIM:**
- This query retrieves the sum of all sales per salesperson, where the sales sum is greater than 2000.

**NOTES:**
- HAVING acts like a WHERE clause on GROUP BY results.
- HAVING is different in that it filters only aggregated results.

```SQL
SELECT 
    salesperson_name,
    SUM(sales) sales_total
FROM 
    sales
GROUP BY 
    salesperson_name
HAVING 
    SUM(sales) < 2000
```

--

**QUERY AIM:**
- This query retrieves the MAX list_price and MIN list_price for each category_id where the MAX is less than 4000 but the MIN is greater than 500.

```SQL
SELECT
    category_id,
    MAX (list_price) max_list_price,
    MIN (list_price) min_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    MAX (list_price) > 4000 OR MIN (list_price) < 500
```

--

**QUERY AIM:**
- This query gets the average list_price for each category_id where the average list price is between 500 and 1000.

```SQL
SELECT
    category_id,
    AVG (list_price) avg_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    AVG (list_price) BETWEEN 500 AND 1000
```

--

## 6c. Grouping Sets

**QUERY AIM:**
- This query retrieves the total sales per brand and category, brand only, category only and finally neither brand nor category (total sales).

**NOTES:**
- We use GROUPING SETS to retrieve multiple differently grouped aggregate totals in one query result.
- As shown below, CUBE and ROLLUP provide shorter ways to declare GROUPING SETS.

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    GROUPING SETS (
        (brand, category),
        (brand),
        (category),
        ()
    )
ORDER BY
    brand,
    category
```

--

## 6d. Cube

**QUERY AIM:**
- This query uses CUBE to get all possible combinations of aggregated results for d1,d2 and d3.

**NOTES:**
- CUBE would return 8 grouping sets as follows:
- d1+d2+d3, d1+d2, d1+d3, d2+d3, d1, d2, d3, NONE (Table total)

```SQL
SELECT
    d1,
    d2,
    d3,
    aggregate_function (c4)
FROM
    table_name
GROUP BY
    CUBE (d1, d2, d3)
```

--

**QUERY AIM:**
- This query retrieves only the sum of sales for brand + category and brand.

**NOTES:**
- This is called a partial cube.
- 'brand' is outside of the CUBE, so is considered the top of the hierarchy.

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    CUBE(category)
```

--

## 6e. Roll Up

**QUERY AIM:**
- This query aims to get the sum of sales for brand + category, brand and overall sales.

**NOTES:**
- ROLLUP assumes a hierarchy - the first column passed in is treated as the top of the hierarchy and every combination of results with 'brand' will be returned.

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP(brand, category)
```

--

**QUERY AIM:**
- This query aims to return sales totals for category + brand, category only and total sales.

```SQL
SELECT
    category,
    brand,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP (category, brand)
```

--

**QUERY AIM:**
- This query retrieves sales totals for brand + category and brand but not full table sales totals as it as a partial ROLLUP.

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    ROLLUP (category)
```

-----

# 7. SubQuery

## 7a. Overview

**QUERY AIM:**
- This query gets order information for all customers based in New York.

**NOTES:**
- The sub query finds all customers in New York. The outer query then gets order details where the customer_id appears in the sub-query.
- We need access to both the orders table and the customers table to retrieve the information we require.

```SQL
SELECT
    order_id,
    order_date,
    customer_id
FROM
    sales.orders
WHERE
    customer_id IN (
        SELECT
            customer_id
        FROM
            sales.customers
        WHERE
            city = 'New York'
    )
ORDER BY
    order_date DESC
```

--

**QUERY AIM:**
- This query retrieves the highest list price item from every order.

**NOTES:**
- The subquery searches the order_items table for every order in the orders table.
- This is an example of how a subquery can be used as a field.

```SQL 
SELECT
    order_id,
    order_date,
    (
        SELECT
            MAX (list_price)
        FROM
            sales.order_items i
        WHERE
            i.order_id = o.order_id
    ) AS max_list_price
FROM
    sales.orders o
ORDER BY order_date desc
```

--

## 7b. Correlated SubQuery

**QUERY AIM:**
- This query finds the products with the highest list price in their category.

**NOTES:**
- Correlated subqueries rely on the outer table with a  column match.
- The table in the subquery can be the same table as the outer table (using table aliases) or a different table.

```SQL
SELECT
    product_name,
    list_price,
    category_id
FROM
    production.products p1
WHERE
    list_price IN (
        SELECT
            MAX (p2.list_price)
        FROM
            production.products p2
        WHERE
            p2.category_id = p1.category_id
        GROUP BY
            p2.category_id
    )
ORDER BY
    category_id,
    product_name
```

--

## 7c. Exists

**QUERY AIM:**
- This query returns all customers if they have placed more than 2 orders.

**NOTES:**
- For every customer_id, the subquery counts up the number of orders placed for that customer_id, returning True if more than 2, or False if less.

```SQL
SELECT
    customer_id,
    first_name,
    last_name
FROM
    sales.customers c
WHERE
    EXISTS (
        SELECT
            COUNT (*)
        FROM
            sales.orders o
        WHERE
            customer_id = c.customer_id
        GROUP BY
            customer_id
        HAVING
            COUNT (*) > 2
    )
ORDER BY
    first_name,
    last_name
```

--

**QUERY AIM:**
- This query finds all orders placed by customers who are based in San Jose.

**NOTES:**
- Sometimes a simpler subquery using IN can achieve the same results, without the need for a correlated subquery.

```SQL
SELECT
    *
FROM
    sales.orders
WHERE
    customer_id IN (
        SELECT
            customer_id
        FROM
            sales.customers
        WHERE
            city = 'San Jose'
    )
ORDER BY
    customer_id,
    order_date
```

--

**NOTES:**
- The same query as above using a correlated subquery and EXISTS.

```SQL
SELECT
    *
FROM
    sales.orders o
WHERE
    EXISTS (
        SELECT
            customer_id
        FROM
            sales.customers c
        WHERE
            o.customer_id = c.customer_id
        AND city = 'San Jose'
    )
ORDER BY
    o.customer_id,
    order_date
```

--

## 7d. Any

**QUERY AIM:**
- This query gets all products and their list price where the product has an order quantity of more than 2 in the order_items table.

**NOTES:**
- Here we return ANY matches where the product_id from the outer query is found inside the subquery.
- The query will return true and stop when ANY matches are found.

```SQL
SELECT
    product_name,
    list_price
FROM
    production.products
WHERE
    product_id = ANY (
        SELECT
            product_id
        FROM
            sales.order_items
        WHERE
            quantity >= 2
    )
ORDER BY
    product_name
```

--

## 7e. All

**QUERY AIM:**
- This query finds all products that have a higher list_price than ALL of the average list prices for each brand_id.

**NOTES:**
- Every product is assessed and the list_price of the product must be higher than ALL of the averages from the subquery to be returned.

```SQL
SELECT
    product_name,
    list_price
FROM
    production.products
WHERE
    list_price > ALL (
        SELECT
            AVG (list_price) avg_list_price
        FROM
            production.products
        GROUP BY
            brand_id
    )
ORDER BY
    list_price
```

-----

# 8. Set Operators

**QUERY AIM:**
- This query aims to get a complete combined list of all staff members and customers without any duplicates.

**NOTES:**
- UNION will remove any duplicates.
- The data types must be the same in the two tables we assess using set operators.

## 8a. Union

```SQL
SELECT
    first_name,
    last_name
FROM
    sales.staffs
UNION
SELECT
    first_name,
    last_name
FROM
    sales.customers
```

--

## 8b. Union All

**QUERY AIM:**
- This query aims to get a combined list of all staff and customers without removing any duplicates.

**NOTES:**
- UNION ALL does NOT remove duplicates.
- Any staff members who are also customers would be included twice.

```SQL
SELECT
    first_name,
    last_name
FROM
    sales.staffs
UNION ALL
SELECT
    first_name,
    last_name
FROM
    sales.customer
```

--

## 8c. Intersect

**QUERY AIM:**
- In this query, we want to find the players who appear in both tables.

**NOTE:**
- The player of the year tables are invented and have a player_id and player_name column only.

**NOTES:**
**UNION vs. INTERSECT**
- INTERSECT here will only return players who appear in BOTH tables.
- UNION will find players who appear in EITHER table (but with no repetition of players)

```SQL
SELECT 
    player_id, 
    player_name
FROM 
    player_of_the_year_2021
INTERSECT
SELECT 
    player_id,
    player_name
FROM 
    player_of_the_year_2020
```

--

## 8d. Except

**QUERY ORDER:**
- This query retrieves employees who do not appear in the completed_health_and_safety table.

**NOTES:**
- EXCEPT assesses if people from the employees table appear in the second table, not vice-versa.
- The completed_health_and_safety table is invented and simply has first_name and last_name columns of employees who have taken a health and safety course.

```SQL
SELECT 
    first_name,
    last_name
FROM
    employees
EXCEPT
SELECT
    first_name,
    last_name
FROM 
    completed_health_and_safety
```

-----

# 9. Common Table Expressions

## 9a. CTEs

**QUERY AIM:**
- This query returns the sales totals per salesperson per year for the year 2019.

**NOTES:**
- In the WITH statement, alias column names are optional.
- A CTE can be a clearer way to write subqueries, among other uses.

```SQL
WITH sales_person_yearly_total (sales_person_id, sales, year) AS (
SELECT
    id,
    SUM(sales_total)
    YEAR(sales_date)
FROM 
    sales
GROUP BY 
    id,
    YEAR(sales_date)
)
SELECT 
    sales_person_id,
    sales,
    year
FROM 
    sales_person_yearly_total
WHERE
    year = 2019

```

--

**QUERY AIM:**
- This query returns the average order count per member of staff for the year 2018.


**NOTES:**
- You don't need to create column aliases inside WITH statement, as shown below.
- The cte_sales table returns the number of orders per staff member (for 2018) which is then used to calculate the average number of orders from.

```SQL
WITH cte_sales AS (
    SELECT 
        staff_id, 
        COUNT(*) order_count  
    FROM
        sales.orders
    WHERE 
        YEAR(order_date) = 2018
    GROUP BY
        staff_id
)
SELECT
    AVG(order_count) average_orders_by_staff
FROM 
    cte_sales
```

--

**QUERY AIM:**
- This query demonstrates how you can create multiple CTE tables and use both tables in a join, or any other SQL statement.

```SQL
WITH cte_1 AS (
SELECT ....
),
WITH cte_2 AS (
SELECT ...
)
SELECT 
    col_a,
    col_b
FROM 
    cte_1
    INNER JOIN cte_2 ON...
WHERE...
ORDER BY...

```

--

## 9b. Recursive CTEs

**QUERY AIM:**
- This query simply starts with 1 and keeps counting up to 4, returning 1 to 4.

**NOTES:**
- Recursive CTE queries refer to the original cte itself as a starting point, then loops until a condition is met.

```SQL
WITH cte ( value )
  AS (
       SELECT   1
       UNION ALL
       SELECT   value + 1
       FROM     cte
       WHERE    value < 4
     )
SELECT  *
FROM    cte
```

--

**QUERY AIM:**
- This query returns all staff members and their manager, also returning the staff members who do not have a manager (NULL).

**NOTES:**
- Recursive CTEs often query hierarchical data.
- The starting point in the query below is the top manager in the hirearchical structure (manager_id IS NULL)
- Using the first result (main manager) as a base, we then use UNION ALL and then retreive the employees under the main manager. Then we get the employees under the employees under the manager. Then we get the employees under the employees under the employees under the manager and so on.

- The cte table in recursive CTEs always calls itself!

```SQL
WITH manager AS
(
SELECT 
    staff_id,
    first_name + ' ' + last_name full_name,
    manager_id
FROM
    sales.staffs
WHERE
    manager_id IS NULL
UNION ALL
SELECT 
    e.employee_id,
    e.first_name + ' ' + e.last_name full_name, 
    e.manager_id
FROM
    sales.staffs e
    INNER JOIN manager
    ON m.staff_id = e.manager_id
)
SELECT *
FROM 
manager
```

--

**QUERY AIM:**
- This query returns all items and the product that they belong to.

**NOTES:**
- This is another example of a hierarchical query using a recursive CTE.
- For this example, imagine a car being the main product. The steering wheel, seats and doors would all be items belong to that car.
- Smaller items such as the handle could be an item that belongs to the car door.

```SQL
WITH outer_products AS
(
SELECT 
    item_id,
    item_name,
    product_id
FROM
    products
WHERE
    product_id IS NULL
UNION ALL
SELECT 
    p.item_id,
    p.item_name, 
    p.product_id
FROM
    products p
    INNER JOIN outer_products o
    ON o.item_id = p.product_id
)
SELECT *
FROM 
outer_products
```

-----

# 10. Pivot

## 10a. Pivot

**QUERY AIM:**
- This query retrieves the total number of products per category name, with the category names pivoted to columns created with the PIVOT command.

**NOTES:**
- The inner query uses an INNER JOIN to retrieve the category name for each product.
- In this example we use FOR and IN inside PIVOT to specify the category names we want.
- The categories then become the columns in our result, with the total number of products displayed for each category.

CatA, CatB, CatC, CatD

25, 34, 42, 59

```SQL
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id
    FROM 
        products p
        INNER JOIN categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [CatA], 
        [CatB], 
        [CatC], 
        [CatD]
) AS pivot_table
```

--

**QUERY AIM:**
- This query retrieves the total number of products per category name, per model year, with the category names pivoted to columns using the PIVOT command, and the years added as rows.

**NOTE:**
- This query is the same as the one above, except that we add model_year to the query.
- By adding another column (model_year), we add a 'row group' which means the year becomes a row header, allowing us to see product totals per category per year.

model_year, CatA, CatB, CatC, CatD

2019, 3, 50, 22, 6

2020, 52, 4, 12, 13

2021, 41, 5, 29, 82

```SQL
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id,
        model_year
    FROM 
        products p
        INNER JOIN categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [CatA], 
        [CatB], 
        [CatC], 
        [CatD]
) AS pivot_table
```

-----

# 11. Expressions

## 11a. Case

**QUERY AIM:**
- This query returns the sum of all scores marked either 'Pass' or 'Fail'.

**NOTES:**
- This query uses CASE to assess each test_score and add 1 or 0 to the SUM column 'Pass' or 1 or 0 the SUM column 'Fail'.
- The test_scores table is invented and simply has a test_score column.

```SQL
SELECT 
    SUM(CASE
        WHEN test_score > 70
        THEN 1
        ELSE 0
    END) AS 'Pass',
    SUM(CASE
        WHEN test_score <= 70
        THEN 1
        ELSE 0
    END) AS 'Fail'
FROM 
    test_scores
```

--

**QUERY AIM:**
- This query returns all student names and score along with an additional outcome column determined by the score they managed on the test.

**NOTES:**
- The test_scores table is invented and only has a name and score column.

```SQL
SELECT 
    name, 
    score,
    CASE
        WHEN score > 70
            THEN 'Pass'
        WHEN score <= 70 AND score >= 60
            THEN 'Possible retake'
        WHEN score <60 
            THEN 'Fail'
    END outcome
FROM 
    test_scores
```

--

## 11b. Coalesce

**QUERY AIM:**
- This query retrieves the first name, last name and phone number of all employees, however if the phone number is NULL, then 'N/A' is returned instead.

**NOTES:**
- COALESCE will choose the first non null value that is passed in.

```SQL
SELECT 
    first_name,
    last_name,
    COALESCE(phone,'N/A')
FROM 
    employees
```

--

**QUERY AIM:**
- This query aims to retrieve all staff and a monthly salary rate for each employee.

**NOTES:**
- COALESCE finds the first NON NULL value and performs the given calculation (or no calculation if the employee has a monthly_rate.)
- In this example all staff have one of hourly rate, weekly rate OR monthly rate. The others will be NULL.  
- This is a clever way to achieve a uniform result for all staff salaries.

```SQL
SELECT
    staff_id,
    COALESCE(
        hourly_rate*22*8, 
        weekly_rate*4, 
        monthly_rate
    ) monthly_salary
FROM
    salaries
```

--

**NOTES:**
- The action of finding the first non null value can also be achieved with CASE and WHEN, THEN...
- COALESCE is a tidier way of finding the first non null value.

```SQL
COALESCE(e1,e2,e3)
......
CASE
    WHEN e1 IS NOT NULL THEN e1
    WHEN e2 IS NOT NULL THEN e2
    ELSE e3
END
```

--

# 11c. Null If

**QUERY AIM:**
- This query aims to find clients whose email is blank or NULL.

**NOTES:**
- NULLIF returns NULL if the first parameter is equal to the second parameter.
- In this example we specifically want NULL returned instead of empty strings.

```SQL
SELECT 
    name,
    email
FROM 
    clients
WHERE
    NULLIF(email, '') IS NULL

```

--

**NOTES:**
- The concept of checking if the 1st parameter is equal to the 2nd parameter can be replicated by using CASE.

```SQL
SELECT 
    NULLIF(a,b)
........ equal to .....
CASE
    WHEN a=b
    THEN NULL
END
```